In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [3]:
def angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [5]:
cap = cv2.VideoCapture(0)
counter = 0 
stage = None
k = 0
with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence = 0.5) as pose:
    while cap.isOpened():
        ret , frame = cap.read()
        #image pre processing________________________________________________________________________________________
        img = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
        #processing__________________________________________________________________________________________________
        result = pose.process(img)
        #reverting the image back to original state__________________________________________________________________
        img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        img.flags.writeable = True
        
        #Taking landmarks and finding angles__________________________________________________________________________
        try:
            landmarks = result.pose_landmarks.landmark
            shoulder  = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow     = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist     = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
                         landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            k         = angle(shoulder,elbow,wrist)
            cv2.putText(img, str(k), tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            if k >150 :        
                stage = 'down'
            elif k<50 and stage == 'down' :
                counter = counter +1
                stage = 'up'
                print(counter)   
        except:
            pass
        
        #Rendering , pose_landmarks , pose connections_____________________________________________________ 
        mp_drawing.draw_landmarks(img,result.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color = (245,66,11),thickness = 2,circle_radius = 1),
                                  mp_drawing.DrawingSpec(color = (34,65,31),thickness = 2,circle_radius = 2))
        
        
        #stage and curl progress visualizing_____________________________________________________________________
        end_point = tuple([30,int(k)])
        cv2.rectangle(img, (0,0), end_point, (245,247,16), -1)
        cv2.putText(img, stage, (60,60),cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        #________________________________________________________________________________________________________
        cv2.imshow("counter",img)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()